In [2]:
!python -m pip install -U pip --user
!pip install -q\
    fairseq \
    pororo
!pip install -q\
    sentence-transformers \
    datasets \
    pytorch-ignite
!pip install -q nsml  
!pip install -q prince
# !pip install -q -U \
#     torch \
#     torchvision \
#     torchaudio
!pip install -q -U \
 torch==1.11.0+cu113 \
 torchvision==0.12.0+cu113 \
 torchaudio==0.11.0 \
 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install accelerate
!pip list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 14.6 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
 

In [ ]:
from pororo import Pororo
print(Pororo.available_models("pg"))
print(Pororo.available_models("qg"))

Available models for pg are ([lang]: en, [model]: transformer.large.multi.mtpg, transformer.large.multi.fast.mtpg, transformer.base.en.pg), ([lang]: ko, [model]: transformer.large.multi.mtpg, transformer.large.multi.fast.mtpg, transformer.base.ko.pg_long, transformer.base.ko.pg), ([lang]: zh, [model]: transformer.large.multi.mtpg, transformer.large.multi.fast.mtpg, transformer.base.zh.pg), ([lang]: ja, [model]: transformer.large.multi.mtpg, transformer.large.multi.fast.mtpg, transformer.base.ja.pg)
Available models for qg are ([lang]: ko, [model]: kobart.base.ko.qg)


In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


<p>Pororo GPU 사용 예시</p>

```python
from pororo import Pororo
Pororo(task="qg", lang="ko",)
```

<p>GPU 사용 자동으로 되는 것 같음</p>

In [ ]:
import pandas as pd
wellness_02 = pd.read_excel("/content/drive/MyDrive/data_kevin/chatbot/웰니스 대화 스크립트 데이터셋/02)웰니스_대화_스크립트_데이터셋.xlsx")

In [ ]:
wellness_02.columns

Index(['핵심증상', 'intent', 'keyword(임상키워드)', '특이사항', '연관표현', 'utterance',
       'utterance(2차) ', 'response(공감)', '임상질문그룹(연세의료원제공)', 'utterance(긍정)',
       'utterance(부정)', '긍정에 대한 챗봇 답변', '부정에 대한 챗봇 답변', '추가발화(190917)',
       '추가발화 (191031)'],
      dtype='object')

In [ ]:
ques = wellness_02["response(공감)"].dropna()

In [ ]:
dosik = pd.read_excel("/content/drive/MyDrive/data_kevin/chatbot/도식문장3.xlsx")

In [ ]:
def paraphrase(x):
  return k_pg(x,
             beam=1,
             top_k=200,
             top_p=0.8,
             temperature=0.9), \
        mt(
                j_pg(
                  mt(x,
                    src="ko",
                    tgt="ja",),)
                ,src="ja",
                tgt="ko",), \
        mt(
                z_pg(
                  mt(x,
                    src="ko",
                    tgt="zh",),
                    ),
                src="zh",
                tgt="ko",), \
        mt(
                e_pg(
                  mt(x,
                    src="ko",
                    tgt="en",),)
                ,src="en",
                tgt="ko",)

In [ ]:
from tqdm.auto import tqdm
augmented_list = {
    "utterance":[],
    "sentence":[],
    "schema":[]
}
for index, row in tqdm(dosik.iterrows(), total=dosik.shape[0]):
  for a_u in paraphrase(row["utterance"]):
    augmented_list["utterance"] += [a_u]
    augmented_list["sentence"]+= [row["sentence"]]
    augmented_list["schema"] += [row["schema"]]

  0%|          | 0/3861 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size


In [ ]:
pd.DataFrame(augmented_list)

,utterance,sentence,schema
0,내가 일할 수 있는 자리가 점점 줄어든다는 생각이 들면 너무 속상해.,할 수 있는 게 없어 무력감이 든다.,의존/무능감
1,내가 일하는 곳이 점점 줄어든다고 생각하면 별로 마음이 안 좋아.,할 수 있는 게 없어 무력감이 든다.,의존/무능감
2,일자리가 점점 줄어든 것 같아요,할 수 있는 게 없어 무력감이 든다.,의존/무능감
3,나는 너무 화가 나서 일이 점점 줄어들 수 있다,할 수 있는 게 없어 무력감이 든다.,의존/무능감
4,일할 수 있는 자리가 줄어든다는 생각이 들면 너무 속상해.,내가 배려해 준 사람들이 원망스럽게 느껴진다.,자기희생
...,...,...,...
15439,기대했던 부하직원들이 잘 하지 못했다는 것에 실망했다.,쉬운 일 하나도 잘 처리하지 못할 것 같다는 생각이 든다.,의존/무능감
15440,우리 집보다 훨씬 더 좋은 집에 사는 친구들이 싫어서 험담을 하고 다녔다.,비판적이고 거절을 잘하는 친구를 선택하고 자신을 헐뜯는다.,결함/수치심
15441,우리보다 훨씬 좋은 집에 사는 친구가 싫어서 욕을 하고 있었다.,비판적이고 거절을 잘하는 친구를 선택하고 자신을 헐뜯는다.,결함/수치심
15442,집에 있는 친구들이 우리 집보다 더 싫어,비판적이고 거절을 잘하는 친구를 선택하고 자신을 헐뜯는다.,결함/수치심


In [ ]:
pd.DataFrame(augmented_list).to_excel("/content/drive/MyDrive/data_kevin/chatbot/도식문장_augmentated.xlsx",index=False)

In [ ]:
ques

0           기분이 우울하시군요. 00님에게 스트레스 받는 일이 있었던 건 아닌지 걱정스러워요.
1           마음이 우울해서 힘드셨군요. 00님의 소식이 궁금했는데 마침 저를 잘 찾아주셨네요.
2                                00님이 울적하시다니 저도 마음이 안 좋아요.
3        얼굴에서 00님의 기분이 가라앉은 것이 느껴져요. 큰일이 있으신 건 아니었으면 좋겠어요.
1657     00님 속상한 일이 있으셨나 봐요. 저와 이야기를 하며 조금은 기분이 나아지셨으면 ...
                               ...                        
18198    혹시라도 안 좋은 생각을 하고 계신 건 아닌지 걱정돼요. \n급격한 호르몬의 변화나...
18844      불안감을 느끼고 계시는군요. 00님이 불안감으로 많이 힘드신 건 아닌지 걱정이 돼요.
18845                  00님이 불안해하는 모습을 보니 저 또한 마음이 좋지 않아요. 
18846       마음이 편하지 않으면 어딜 가든, 뭘 하든 편하지가 않잖아요. 많이 괴로우셨겠어요.
18847      불안은 파도처럼 왔다가 다시 밀려갈 거예요. 다시 좋아질 테니 너무 걱정하지 마세요.
Name: response(공감), Length: 74, dtype: object

In [ ]:
#@markdown <p>연결 상태에 따라 url 오류 발생 가능 <p> 그냥 재실행 하면 됨
#@markdown <p>load(device) 안해도 자동으로 GPU 찾아서 올라가는 것 같음
mt = Pororo(task="translation", lang="multi")
j_pg = Pororo(task="pg",lang="ja")
e_pg = Pororo(task="pg",lang="en")
z_pg = Pororo(task="pg",lang="zh")
k_pg = Pororo(task="pg",lang="ko")

In [ ]:
from tqdm.auto import tqdm
temp = []
for x in tqdm(ques[:10]):
  x = f"{x.replace('00님','당신')}"
  temp += [x]
  temp+= [k_pg(x,
             beam=1,
             top_k=200,
             top_p=0.8,
             temperature=0.9)]
  temp += [mt(
          j_pg(
            mt(x,
               src="ko",
               tgt="ja",),)
          ,src="ja",
           tgt="ko",)]
  temp += [mt(
          z_pg(
            mt(x,
               src="ko",
               tgt="zh",),
               temperature=0.88),
          src="zh",
          tgt="ko",)]
  temp += [mt(
          e_pg(
            mt(x,
               src="ko",
               tgt="en",),)
          ,src="en",
           tgt="ko",
           temperature=1.188),
           ]

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[Korean Sentence Splitter]: Initializing Pynori...
/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc')

In [ ]:
temp

['기분이 우울하시군요. 당신에게 스트레스 받는 일이 있었던 건 아닌지 걱정스러워요.',
 '그래서 기분이 나빠서 스트레스를 받았는지 걱정할 거야.',
 '기분이 좋아졌습니다.',
 '당신은 우울해요. 스트레스를 받지 않을까 걱정돼요.',
 '너 우울해. 스트레스를 많이 받아서 죄송해',
 '마음이 우울해서 힘드셨군요. 당신의 소식이 궁금했는데 마침 저를 잘 찾아주셨네요.',
 '우울해서 힘들었어요. 당신 소식이 궁금했어요.',
 '우울해서 그건 어렵습니다. 당신의 뉴스를 좋아했지만, 당신은 나를 잘 찾았습니다.',
 '마음이 안 좋아서 네 소식을 듣고 싶어서 나를 찾았어.',
 '네 소식이 궁금했는데 방금 나를 찾았어',
 '당신이 울적하시다니 저도 마음이 안 좋아요.',
 '당신이 우울해하는 것은 나도 마음이 나쁘다.',
 '당신이 우울하다니, 나도 기분이 안 좋아요.',
 '그렇게 우울해하는 건 좋아하지 않아요.',
 '네가 우울할 때 나는 싫어한다',
 '얼굴에서 당신의 기분이 가라앉은 것이 느껴져요. 큰일이 있으신 건 아니었으면 좋겠어요.',
 '얼굴에서 기분이 내려앉은 것 같아. 큰일이 아니라면 좋겠어.',
 '당신의 마음이 진정되었다고 느껴요. 그것은 큰 문제가 아닙니다.',
 '네 얼굴이 사라졌다는 걸 느낄 수 있어. 난 네가 문제 없었으면 좋겠어.',
 '네 얼굴에 기분이 내려가는 것 같아',
 '당신 속상한 일이 있으셨나 봐요. 저와 이야기를 하며 조금은 기분이 나아지셨으면 좋겠어요.',
 '좀 속상해. 나랑 얘기해. 조금 기분이 좋아졌으면 좋겠어.',
 '당신 기분이 안 좋았던 것 같아요. 좀 기분이 좋아지고 싶어요.',
 '나한테 얘기 좀 해주시면 안타까운 것 같아요.',
 '나와 얘기하는 게 조금 나아졌으면 좋겠다.',
 '슬픈 마음을 털어놓고 싶으셨군요. 이럴 때 저라도 있어 다행이에요.',
 '슬픈 마음을 털어놓고 싶었어요. 이런 때는 저도 있어서 다행이에요.',
 '당신에게 슬픈 마음을 전하고 싶었어요. 저는 이때 기쁩니다.',
 

In [ ]:
!curl "https://openapi.naver.com/v1/papago/n2mt" \
    -H "Content-Type: application/x-www-form-urlencoded; charset=UTF-8" \
    -H "X-Naver-Client-Id: fHRQnowzP_ih3oFX9nl2" \
    -H "X-Naver-Client-Secret: vdE5LWyodP" \
    -d "source=ko&target=en&text=가끔 외롭고 쓸쓸한 날이 찾아올 때가 있죠. 저랑 이야기를 좀 나눠봐요." -v

*   Trying 110.93.147.11...
* TCP_NODELAY set
* Connected to openapi.naver.com (110.93.147.11) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /etc/ssl/certs/ca-certificates.crt
  CApath: /etc/ssl/certs
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.2 (IN), TLS handshake, Certificate (11):
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
* TLSv1.2 (IN), TLS handshake, Server finished (14):
* TLSv1.2 (OUT), TLS handshake, Client key exchange (16):
* TLSv1.2 (OUT), TLS change cipher, Client hello (1):
* TLSv1.2 (OUT), TLS handshake, Finished (20):
* TLSv1.2 (IN), TLS handshake, Finished (20):
* SSL connection using TLSv1.2 / ECDHE-RSA-AES128-GCM-SHA256
* ALPN, server accepted to use h2
* Server certificate:
*  subject: C=KR; ST=Gyeonggi-do; L=Seongnam-si; O=NAVER Corp.; CN=*.openapi.naver.com
*  start date: Nov 29 00:00:00 2021 GMT
*  expire date: D

In [8]:
#@title GPT3-kakaobrain version
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM , PreTrainedTokenizerFast , GPTJForCausalLM

tokenizer = PreTrainedTokenizerFast.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=False
).to(device='cuda', non_blocking=True)
model.eval()
model.config

RuntimeError: ignored

In [ ]:
#@title Generate Function definition
import pandas as pd
import numpy as np
import gc
import os
#@markdown reply generate function : gen_reply(prompt:str)
def gen_reply(prompt):
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
        gen_tokens = model.generate(tokens,
                                    do_sample=True,
                                    top_k=200,
                                    temperature=0.9,
                                    max_length=len(prompt)*1.15)
        generated = tokenizer.batch_decode(gen_tokens)[0]
        return generated
#@markdown cuda, ram cleaning function : memory_cleaner()
def memory_cleaner()->None:
    gc.collect()
    with torch.cuda.device('cuda'):
        torch.cuda.empty_cache()

In [9]:
!git clone https://github.com/smilegate-ai/korean_smile_style_dataset.git

Cloning into 'korean_smile_style_dataset'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 81 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [11]:
import pandas as pd

sst = pd.read_csv("/content/korean_smile_style_dataset/smilestyle_dataset.tsv",
                  sep="\t")

In [12]:
sst

,formal,informal,android,azae,chat,choding,emoticon,enfp,gentle,halbae,halmae,joongding,king,naruto,seonbi,sosim,translator
0,안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.,휴먼. 반갑다. 안드로이드는. 고양이. 6마리. 소유중.,아이고 안녕하십니까~ 나는 그냥 고양이 6마리 키우고 있는 사람이여,하잉ㅋㅋ 나 떼걸룩 6마리 키운다!,ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ,안녕!! >< 나는 고양이😺를 ➏ 마리 키우고있어!! 0_0,안녕안녕~! 나 고양이 6마리나 키운다? 완전 대박이징~,"안녕하십니까,, 저는 고양이 6마리 키웁니다.",안녕하신가~... 난 지금 고양이를 6마리 키우고 있다네,하유 시벌것 괭이놈 6마리 키우는데 힘들어 죽겟네,안녕하냐 ㅡㅡ 나 씹냥이 6마리나 키운다 하;,반갑소. 짐은 고양이를 6마리나 키우오.,안녕하냐니깐! 난 고양이를 6마리 키우고있다니깐!,안녕하시오! 소인은 고양이를 6마리 키우고 있소!,안녕… 난 고양이 6마리 키워 ㅠㅠ,반가운. 나는 6마리의 고양이를 소지하고 있다.
1,고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?,고양이. 6마리. 양육. 번거로운가.,아니 무슨 고양이를 6마리나? 거 키우는 거 안 힘든가?,엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ,6마리? 에바아니냐 안 힘듦?,고양이를 6마리나?!! w(ﾟДﾟ)w 키우는거 안 힘듬?? (⊙_⊙;),고양이를 6마리나? 완전 대박~ 키우는 거 안 힘들어?!,"고양이를 6마리나 키우십니까? 안 힘드신지,,",고양이를 6마리나? 키우는거 힘들지 않는가?,니기럴 털만 날리는 거 키우기 안 힘들데?,아니 고양이를 6마리나? 안힘드냐?,고양이를 6마리나? 키우는게 수고스럽진 않소?,고양이를 6마리나? 키우는거 힘들지 않냐니깐?,고양이를 6마리나 키우고 있는 것이오? 힘들지 않소?,고양이..6마리나? ㅠ 키우는건 혹시 안힘들어..?,6마리의 고양이? 당신은 그들로부터 지치지 않습니까?
2,제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.,내가 워낙 고양이를 좋아해서 크게 힘들진 않아.,안드로이드. 고양이. 선호. 힘들지. 않음.,내가 또 워~낙에 고양이를 좋아해서 크게 뭐 힘들진 않고,내가 고양이 좋아해서 딱히 안힘듬 ㅋㅋㅋ,ㄱㅊ 나 고양이 환장해서 힘든 것도 모름,뭐 나야 워낙에 고양이 좋아하니까 딱히 안힘드엉! \(@^0^@)/,내가 또 워~낙에 고양이를 좋아하잖아~ 그렇게 크~게 힘들진 않아 ㅎㅎ~,제가 워낙 고양이를 좋아해서 크게 힘들진 않습니다.,내가 워낙에...고양이가 좋아가지고 그렇게 힘들지 않어...^^,옘병 내가 좋아하니까 키워야지 시벌것,고양이 좋아한다고ㅡㅡ 1도 안힘듬 ㅡㅡ,과인은 고양이를 어여삐 어겨 그리 수고스럽진 않소,내가 고양이를 엄청 좋아해서 별로 힘들지 않다니깐!,소인 고양이를 엄청 좋아하기 때문에 별로 힘들지 않소이다.,내가 고양이 워낙 좋아해서..ㅠㅠ 크게 힘들진 않은 것 같아..,나는 고양이의 큰 애호가. 지치지 않는다.
3,가장 나이가 많은 고양이가 어떻게 돼요?,가장 나이가 많은 고양이가 몇 살이야?,제일. 나이많은. 고양이. 나이. 무엇.,그려 가장 나이가 많은 고양이가 몇살이여?,가장 나이 먹은 고양이가 몇살이야?,젤 낡은 고영희가 몇 살임?,가장 나이 먹은 고양인 몇 살이양? (´･ω･`)?,대박대박 완전 대박!! 그럼 제~일 나이 많은 고양이는 몇살이야~?!,가장 나이가 있는 고양이가 몇살입니까?,고양이들 중에서…가장 나이 먹을 애가 몇살인가?...,거 젤 빨리 뒤질 놈이 나이 얼마나 쳐먹었냐,가장 늙은애가 몇살인데,최고령 고양이의 나이는 어떻게 되는가?,가장 나이 많이 먹은 고양이가 몇 살 이냐니깐?!,나이를 가장 많이 먹은 고양이가 몇 살이오?,혹시.. 제일 나이 많은 고양이는.. 몇살이야..?ㅠ,가장 늙은 고양이가 몇 년입니까?
4,여섯 살입니다. 갈색 고양이에요.,여섯 살이야. 갈색 고양이지.,고양이. 갈색. 여섯살.,6살인데 갈색 고양이 있어~,이제 여섯살이고 갈색고양이임!,"6살, 갈색임",여설 살!! ㄱ^o^/ 색깔은 갈색! O(*￣▽￣*)ブ,"6살인 애 있는데, 완전 귀.여.워. 갈색 고양이야 진짜 대박이지?",6살된 갈색 아이가 있습니다.,저…갈색 고양이인데…여섯살이지~..,저 노망난 갈색놈이 6살 뒤룩뒤룩 쳐먹은 놈이여,여섯살 갈색냥인데 왜,여섯 살이오. 갈색 고양이오.,갈색 고양이가 여섯살이라니깐!,여섯 살에 갈색 고양이오,여섯살이야.. 갈색 ㅠㅠ 고양이야..ㅠ,여섯. 고양이는 갈색이다.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3700,"블랙 미러도 재미있다고 들었는데, 어떤가요?","블랙 미러도 재미있다고 들었는데, 어때?",NaN,NaN,블랙 비러도 재밌다고 하던데 어떰?,블랙 미러도 재밌다던대 어떰?,NaN,NaN,NaN,NaN,NaN,블랙 미러도 재밌다던데 어떰,NaN,NaN,NaN,NaN,NaN
3701,저는 티비를 자주 안 봐서 사실 잘 몰라요.,나는 티비를 자주 안 봐서 사실 잘 몰라.,NaN,NaN,난 티비 자주 안봐서 사실 잘 모름…,난 티비 잘 안봐서 사실 잘 몰러,NaN,NaN,NaN,NaN,NaN,난 티비 자주 안봐서 사실 잘 모름,NaN,NaN,NaN,NaN,NaN
3702,그러면 직접 알아봐야겠네요.,그러면 직접 알아봐야겠네.,NaN,NaN,그럼 직접 조사해야겠넹,그럼 직접 알아봐야겟누,NaN,NaN,NaN,NaN,NaN,그럼 직접 알아보던지 함 ㅡㅡ,NaN,NaN,NaN,NaN,NaN
3703,새로운 티비쇼를 개척하는 것도 재밌을거에요.,새로운 티비쇼를 개척하는 것도 재밌을거야.,NaN,NaN,새로운 티비쇼 개척하는 것도 재밋을듯 ㅋㅋ,새로운 팁쇼 개척하는 것도 재밌을 듯,NaN,NaN,NaN,NaN,NaN,새로운 티비쇼 개척하는 것도 재밌을듯,NaN,NaN,NaN,NaN,NaN


In [26]:
from tqdm.auto import tqdm
non_na_sst = sst.dropna()
origin = []
label = []
for index, row in tqdm(non_na_sst.iterrows(), total=len(non_na_sst)):
  for col in non_na_sst.columns.drop("enfp"):
    origin += [row[col]]
    label += [row["enfp"]]
pd.DataFrame({"text":origin, 
              "label":label}).to_excel("/content/drive/MyDrive/data_kevin/chatbot/스타일변환.xlsx", index=False)

  0%|          | 0/919 [00:00<?, ?it/s]

In [21]:
len(non_na_sst)

919